In [ ]:
import csv
from pprint import pprint
from lxml import etree
from collections import defaultdict
import urllib.request

In [2]:
class ParseBioPax(object):
    """
    Parses Rhea's BioPax file to facilate parsing as xml in Python
    input: BioPax file path
    return: xml object root as self.root, functions for gathering reactions and participants.
    """
    namespaces = {
        'bp': "http://www.biopax.org/release/biopax-level2.owl#",
        'rhea': "http://www.ebi.ac.uk/rhea#",
        'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        'owl': "http://www.w3.org/2002/07/owl#",
        'base':"http://www.ebi.ac.uk/rhea#",
        'xsd': "http://www.w3.org/2001/XMLSchema#"    
    }
    
    def __init__(self, biopax_path=None, ec2rhea_path=None, download_files=False):
        self.biopax_path = biopax_path
        self.ec2rhea_path = ec2rhea_path
        if download_files:
            self.download_ftp_files()
        self.root = self.parse_bio_pax()
        
    def download_ftp_files(self):        
        biopaxFTP = 'ftp://ftp.ebi.ac.uk/pub/databases/rhea/biopax/rhea-biopax_lite.owl.gz'
        urllib.request.urlretrieve(biopaxFTP, self.biopax_path)
        ec2rheaFTP = 'ftp://ftp.ebi.ac.uk/pub/databases/rhea/tsv/rhea2ec.tsv'
        urllib.request.urlretrieve(ec2rheaFTP, self.ec2rhea_path)
        
    def ec_2_rhea_map(self):
        """
        returns dictionary with ecnumber key and list of rhea master ids as the values
        """
        ecmap = {}
        with open(self.ec2rhea_path, 'r') as ec2r:
            csvfile = csv.reader(ec2r, delimiter="\t")
            fieldnames = next(csvfile)
            d = defaultdict(list)
            for line in csvfile:
                if len(line) ==  4:
                    d[line[3]].append(line[2])
            return d
        
    def parse_bio_pax(self):
        """
        create root lxml object
        """
        tree = etree.parse(self.biopax_path)
        return tree.getroot()

In [3]:
class ParseBioPaxReaction(object): 
    """
    Parse BioPax Reaction from Rhea
    input: <BP_object>: the root BioPax lxml object from class ParseBioPax
            <ec_number>: the ec number of the reaction
            <download files> default is False.  Change to True if the files need to be downloaded
    return: python dictionary of annotated reaction
    e.g.:
        {
        'ec_number': '3.1.7.9',
        'name': 'H2O + tuberculosinyl diphosphate <?> (13R)-edaxadiene + diphosphate',
        'products': ['6953', '2106'],
        'reactants': ['5701', '1283']
        }
    """
    
    namespaces = {
        'bp': "http://www.biopax.org/release/biopax-level2.owl#",
        'rhea': "http://www.ebi.ac.uk/rhea#",
        'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        'owl': "http://www.w3.org/2002/07/owl#",
        'base':"http://www.ebi.ac.uk/rhea#",
        'xsd': "http://www.w3.org/2001/XMLSchema#"    
    }
    
    def __init__(self, BP_Object, ec_number):
        self.ec_number = ec_number
        self.root = BP_Object
        self.reaction = None
        self.rxn = {
            'ec_number': ec_number,
            'reactants': [],
            'products': [],
        } 
        
    def get_reaction_by_rhea(self, rheaID):
        """
        get reaction element object from lxlm root using rheaID
        input: rhea master Id
        return: reaction elemement
        """
        query = "bp:biochemicalReaction[@{%s}about='http://identifiers.org/rhea/%s']" % (ParseBioPaxReaction.namespaces['rdf'], rheaID)
        self.reaction = self.root.findall(query, ParseBioPaxReaction.namespaces)[0]
    
    def gather_participants(self, participant):
        """
        Get chebi id of participant
        input: <participant> participant element from BioPax lxml 
        return chebi id
        """
        
        # retrieve phisicalEntityParticipant
        query ='bp:physicalEntityParticipant[@rdf:about="{}"]'.format(participant)
        part = self.root.find(query, ParseBioPaxReaction.namespaces)
        
        # retrieve PHISICAL-ENTITY predicate
        entity = part.find('bp:PHYSICAL-ENTITY', ParseBioPaxReaction.namespaces)
        query2 = 'bp:physicalEntity[@rdf:about="{}"]'.format(entity.attrib.values()[0])
        chebi = self.root.find(query2, ParseBioPaxReaction.namespaces)
        # get xref purl to retrieve chebi id
        xref = chebi.find('bp:XREF', ParseBioPaxReaction.namespaces).attrib.values()[0]
        
        # retrieve chebi id
        query3 = 'bp:unificationXref[@rdf:about="{}"]'.format(xref)
        chebi_out = self.root.find(query3, ParseBioPaxReaction.namespaces).find('bp:ID', ParseBioPaxReaction.namespaces).text
        return chebi_out
        
    def parse_reactants(self):
        """
        find all reactants for reaction element (self.reaction) and use self.gather participants to retrieve chebi id
        return: appends chebi id of reactant to self.rxn['reactants]
        """
        left_side = self.reaction.findall('bp:LEFT', ParseBioPaxReaction.namespaces)
        for left in left_side:
            self.rxn['reactants'].append(self.gather_participants(participant=left.attrib.values()[0]))
        
    def parse_products(self):
        """
        find all products for reaction element (self.reaction) and use self.gather participants to retrieve chebi id
        return: appends chebi id of product to self.rxn['reactants]
        """
        right_side = self.reaction.findall('bp:RIGHT', ParseBioPaxReaction.namespaces)
        for right in right_side:
            self.rxn['products'].append(self.gather_participants(participant=right.attrib.values()[0]))
     
    def reaction_name(self):
        """
        get name of self.reaction and add to self.rxn
        """
        names = (self.reaction.findall('bp:NAME', ParseBioPaxReaction.namespaces))
        self.rxn['name'] = names[0].text

In [4]:
biopax_obj = ParseBioPax(biopax_path='rhea-biopax_lite.owl.gz', ec2rhea_path='rhea2ec.tsv', download_files=True)

In [5]:
ec_numbers = biopax_obj.ec_2_rhea_map()

In [7]:
count = 0
"""
run the script for 10 ec numbers
"""
for ec, rhea_list in ec_numbers.items():
    count += 1
    if count < 10:
        for rhea in rhea_list:
            reaction_obj = ParseBioPaxReaction(BP_Object=biopax_obj.root, ec_number=ec)
            reaction = reaction_obj.get_reaction_by_rhea(rheaID=rhea)
            products = reaction_obj.parse_products()
            reactants = reaction_obj.parse_reactants()
            name = reaction_obj.reaction_name()
            pprint(reaction_obj.rxn)
       
        

{'ec_number': '1.14.11.10',
 'name': "2'-deoxyuridine + 2-oxoglutarate + O2 <?> "
         '2-deoxy-D-ribono-1,4-lactone + CO2 + succinate + uracil',
 'products': ['CHEBI:17281', 'CHEBI:16526', 'CHEBI:30031', 'CHEBI:17568'],
 'reactants': ['CHEBI:15379', 'CHEBI:16450', 'CHEBI:16810']}
{'ec_number': '1.14.13.123',
 'name': '(+)-(R)-germacrene A + H(+) + NADPH + O2 <?> '
         'germacra-1(10),4,11(13)-trien-12-ol + H2O + NADP(+)',
 'products': ['CHEBI:15377', 'CHEBI:58349', 'CHEBI:62090'],
 'reactants': ['CHEBI:15378', 'CHEBI:15379', 'CHEBI:41595', 'CHEBI:57783']}
{'ec_number': '6.3.1.10',
 'name': '(R)-1-aminopropan-2-ol + adenosylcob(III)yrate + ATP <?> '
         'adenosylcob(III)inamide + ADP + H(+) + phosphate',
 'products': ['CHEBI:43474', 'CHEBI:15378', 'CHEBI:2480', 'CHEBI:456216'],
 'reactants': ['CHEBI:58504', 'CHEBI:30616', 'CHEBI:42677']}
{'ec_number': '6.3.1.10',
 'name': '(R)-1-aminopropan-2-yl phosphate + adenosylcob(III)yrate + ATP <?> '
         'adenosylcob(III)inami